Reference


https://selenium-python.readthedocs.io/locating-elements.html

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time 
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import csv
from pandas import Series,DataFrame
import unicodecsv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import os 
os.chdir("D:\\1082\\商業分析\\期末報告\\資料")

In [ ]:
driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
driver.maximize_window()
driver.get('https://www.booking.com/searchresults.zh-tw.html?aid=304142&label=gen173nr-1FCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AEB6AEB-AECiAIBqAIDuALU9Nr1BcACAQ&sid=3f6cf794363d33fd77d5d4513411ebab&tmpl=searchresults&ac_click_type=b&ac_position=0&checkin_month=6&checkin_monthday=1&checkin_year=2020&checkout_month=6&checkout_monthday=3&checkout_year=2020&class_interval=1&dest_id=-2637824&dest_type=city&from_sf=1&group_adults=4&group_children=0&iata=RMQ&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA%2CA%2CA&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=index&src_elem=sb&srpvid=df6f6607d8860025&ss=%E5%8F%B0%E4%B8%AD%E5%B8%82%2C%20%E5%8F%B0%E4%B8%AD%E5%9C%B0%E5%8C%BA%2C%20%E5%8F%B0%E6%B9%BE&ss_raw=%E5%8F%B0%E4%B8%AD%E5%B8%82&ssb=empty&top_ufis=1&rows=25&offset=100')

# First Layer
抓取最外層資訊，ex:價格、飯店名稱

In [ ]:
hotel_info = pd.DataFrame()
page = 1
while page<9:
    print("almost")
    url = '//*[@id="search_results_table"]/div[4]/nav/ul/li[2]/ul/li['+str(page)+']/a/div[2]'
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    '''
    title
    '''
    hotel_title_crawler = soup.select('.sr-hotel__title')
    hotel_title= pd.DataFrame(columns = ['hotel_title'])

    for element in  hotel_title_crawler:
        title =element.text
        #print(title)
        hotel_title = hotel_title.append({'hotel_title': title}, ignore_index=True)
        hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("\n", "")
        hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("在新視窗開啟", "")
    
    
    '''
    location
    '''
    location_crawler = soup.select('.sr_card_address_line a',class_ = "bui-link")
    hotel_location = pd.DataFrame(columns = ['hotel_location'])
    for element in  location_crawler:
        location =element.text
        hotel_location = hotel_location.append({'hotel_location': location}, ignore_index=True)
        hotel_location["hotel_location"] =hotel_location["hotel_location"].str.replace("\n", "")
        hotel_location["hotel_location"] =hotel_location["hotel_location"].str.replace("顯示在地圖上", "")
    

    '''
    #price
    '''
    hotel_price_crawler = soup.select('.prco-ltr-right-align-helper span',class_= 'bui-u-sr-only')
    hotel_price = pd.DataFrame(columns = ['hotel_price'])
    for element in  hotel_price_crawler:
        price =element.text
        hotel_price= hotel_price.append({'hotel_price':price}, ignore_index=True)
        hotel_price = hotel_price.astype(str)
        hotel_price = hotel_price[~hotel_price['hotel_price'].isin(['NT$'])]
        hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("\n", "")
        hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("TWD", "")
        hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("房價", "")
        hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace(",", "")
        #hotel_price = hotel_price.astype(int)


    '''
    #Review
    '''
    hotel_review_crawler = soup.select('.bui-review-score__text',class_= 'bui-review-score__text')
    hotel_review = pd.DataFrame(columns = ['hotel_review'])
    for element in  hotel_review_crawler:
        review =element.text
        hotel_review = hotel_review.append({'hotel_review': review}, ignore_index=True)
        hotel_review["hotel_review"] =hotel_review["hotel_review"].str.replace("則評語", "")
        hotel_review["hotel_review"] =hotel_review["hotel_review"].str.replace(",", "")


    '''
    #Total Review
    '''
    hotel_total_review_crawler = soup.select('.bui-review-score__badge')
    hotel_total_review = pd.DataFrame(columns = ['hotel_total_review'])
    for element in  hotel_total_review_crawler:
        total_review =element.text
        hotel_total_review = hotel_total_review.append({'hotel_total_review': total_review}, ignore_index=True)

    
    
    '''
    #score Review
    '''
    hotel_score_review_crawler = soup.select('.bui-review-score__title')
    hotel_score_review = pd.DataFrame(columns = ['hotel_score_review'])
    for element in  hotel_score_review_crawler:
        score_review =element.text
        hotel_score_review = hotel_score_review.append({'hotel_score_review': score_review}, ignore_index=True)
    
    
    '''
    #free cancel
    '''
    hotel_cancel = pd.DataFrame(columns = ['hotel_cancel'])
    for element in soup.findAll("sup", {"class": "sr_room_reward sr_room_reward--bold e2e-no-prepayment"}):
        cancel =element.text
        hotel_cancel = hotel_cancel.append({'hotel_cancel': cancel}, ignore_index=True) 
    
    
    '''
    #Hotel REF
    '''
    elems = driver.find_elements_by_xpath('//a[@href][@class="hotel_name_link url"]')
    hotel_ref= pd.DataFrame(columns = ['hotel_ref'])
    for elem in elems:
        ref = elem.get_attribute("href")
        hotel_ref  = hotel_ref.append({'hotel_ref': ref }, ignore_index=True) 
    df = pd.DataFrame()
    df = pd.concat([hotel_title,hotel_location,hotel_price,hotel_review,hotel_total_review,hotel_score_review,hotel_cancel,hotel_ref], axis=1)
    
    
    hotel_info =hotel_info.append(df)
    driver.find_element_by_xpath(url).click()
    #driver.execute_script("arguments[0].click();",myelement)
    #print("next")
    time.sleep(20)
    page = page +1
    
driver.close() 


In [ ]:
##第一層資料輸出

hotel_info.to_csv("Booking_0519.txt", sep='\t', index=False)

# Second Layer

第二層資料輸出，ex:詳細地址、房間型號、房內設施

In [ ]:
hotel_all_fac= pd.DataFrame()
#
for i in range(len(hotel_info["hotel_ref"])):
    
    url = str(hotel_info["hotel_ref"].iloc[i])
    print(i)
    driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
    driver.maximize_window()   
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'lxml')    
    try:
        '''
        title
        '''
        hotel_title= pd.DataFrame(columns = ['hotel_title'])

        for element in  soup.findAll("a", {"id": "hp_hotel_name_reviews"}):
            title =element.text
            hotel_title = hotel_title.append({'hotel_title': title}, ignore_index=True)
            hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("\n", "")
        hotel_title =  hotel_title.reset_index(drop=True)

        '''
        Address
        '''
        hotel_address= pd.DataFrame(columns = ['hotel_address'])
        for element in soup.findAll("span", {"class":"js-hp_address_subtitle"}):
            address =element.text
            hotel_address = hotel_address.append({'hotel_address': address}, ignore_index=True)
            hotel_address["hotel_address"] =hotel_address["hotel_address"].str.replace("\n", "")
        hotel_address =  hotel_address.reset_index(drop=True)

        '''
        #hotel_room_type
        '''

        hotel_room_type= pd.DataFrame(columns = ['hotel_room_type'])
        for element in soup.findAll("span", {"class": "m-rs-bed-display__bed-type-name"}):
            room_type =element.text
            hotel_room_type = hotel_room_type.append({'hotel_room_type': room_type}, ignore_index=True)
            hotel_room_type["hotel_room_type"] =hotel_room_type["hotel_room_type"].str.replace("\n", "")
        hotel_room_type =hotel_room_type.drop_duplicates()
        hotel_room_type = hotel_room_type.reset_index(drop=True)      



        '''
        #Hotel Main facilities Type
        ''' 

        hotel_main_fac = pd.DataFrame(columns = ['hotel_main_fac'])
        hotel_main_fac_temp = pd.DataFrame(columns = ['hotel_main_fac'])
        for element in soup.findAll("div", {"class": "bui-spacer--small"}):
            main_fac =element.text
            #print(cancel)
            hotel_main_fac_temp = hotel_main_fac_temp.append({'hotel_main_fac': main_fac}, ignore_index=True) 
            hotel_main_fac_temp["hotel_main_fac"] =hotel_main_fac_temp["hotel_main_fac"].str.replace("\n", " ") 

        hotel_main_fac[['hotel_main_fac']]=hotel_main_fac['hotel_main_fac'].append(pd.DataFrame(hotel_main_fac_temp.iloc[0,:]))
        hotel_main_fac = hotel_main_fac.reset_index(drop=True)



        '''
        #Hotel facilities
        ''' 


        #hotel_facilities_crawler = soup.select('.hprt-facilities-others span',class_='hprt-facilities-facility')
        #hotel_room_crawler = soup.select('.hprt-facilities-others span')
        hotel_facilities_crawler = soup.select('.hprt-facilities-others span',class_='hprt-facilities-facility')
        #hotel_room_crawler = soup.select('.hprt-facilities-others span')
        hotel_facilities= pd.DataFrame(columns = ['hotel_facilities'])
        hotel_fac= pd.DataFrame(columns = ['hotel_facilities'])
        for element in  hotel_facilities_crawler:
            facilities =element.text
            hotel_facilities = hotel_facilities.append({'hotel_facilities': facilities}, ignore_index=True)
            hotel_facilities["hotel_facilities"] =hotel_facilities["hotel_facilities"].str.replace("\n", "")
            hotel_facilities = hotel_facilities.drop_duplicates()
        hotel_fac =hotel_fac.append({'hotel_facilities':hotel_facilities['hotel_facilities'].str.cat(sep=' ')}, ignore_index=True)
        #hotel_facilities =hotel_facilities.sort_values(by="hotel_facilities").T
        hotel_fac= hotel_fac.reset_index(drop=True) 
        
    except:
        pass
        
    hotel_all_fac_use = pd.DataFrame()
    hotel_all_fac_use= pd.concat([hotel_title,hotel_address,hotel_room_type,hotel_main_fac,hotel_fac], axis=1)    
    hotel_all_fac = hotel_all_fac.append(hotel_all_fac_use)
    driver.close()   
    continue

In [ ]:
##第二層資料輸出
hotel_info_fac= pd.DataFrame()
hotel_info_fac= pd.merge(hotel_info,hotel_all_fac,on = 'hotel_title',how = 'left')
hotel_info_fac.to_csv("hotel_info_fac0519.txt", sep='\t', index=False)

## Hotel comment

該飯店評價，分成正面評價與負面評價

In [ ]:
##讀取飯店資訊
hotel_info = pd.read_excel("booking_all_data.xlsx",sheet_name="all_hotel")

In [ ]:
hotel_all_comment =pd.DataFrame()

for i in range(len(hotel_info["hotel_ref"])):
    
    url = str(hotel_info["hotel_ref"].iloc[i])
    hotel_title =str(hotel_info["hotel_title"].iloc[i])
    print(i)
    driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
    driver.maximize_window()   
    driver.get(url)
    driver.find_element_by_xpath('//*[@id="show_reviews_tab"]').click()
    hotel_good_bad_comment =pd.DataFrame()
    hotel_good_review_all = pd.DataFrame(columns = ['hotel_good_review'])
    hotel_bad_review_all = pd.DataFrame(columns = ['hotel_bad_review'])
    for  j in range(1,6):
        try:
            soup = BeautifulSoup(driver.page_source,'lxml')
            comment = '//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[2]/div/div['+str(j+1)+']/a'


            '''
            hotel good comment
            '''
            hotel_good_review = pd.DataFrame(columns = ['hotel_good_review'])
            for element in soup.findAll("div", {"class": "c-review__row"}):
                good_review =element.text
                hotel_good_review= hotel_good_review.append({'hotel_good_review':good_review}, ignore_index=True)
                hotel_good_review["hotel_good_review"] =hotel_good_review["hotel_good_review"].str.replace("\n", "")
                #hotel_good_review["hotel_good_review"] =hotel_good_review["hotel_good_review"].str.replace("不喜歡", "")
                hotel_good_review=hotel_good_review[~hotel_good_review.hotel_good_review.str.contains("不喜歡")]
                #print("good"+good_review)

            hotel_good_review= hotel_good_review.reset_index(drop=True) 
            hotel_good_review_all = hotel_good_review_all.append(hotel_good_review)

            '''
            hotel bad comment
            '''
            
            hotel_bad_review = pd.DataFrame(columns = ['hotel_bad_review'])
            for element in soup.findAll("div", {"class": "c-review__row lalala"}):
                bad_review =element.text
                print("baddddddd"+bad_review)
                hotel_bad_review= hotel_bad_review.append({'hotel_bad_review':bad_review}, ignore_index=True)

                hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("\n", "")
                hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("不喜歡", "")
                hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("·", "")

            hotel_bad_review= hotel_bad_review.reset_index(drop=True) 
            hotel_bad_review_all = hotel_bad_review_all.append(hotel_bad_review)

            if j == 5:
                pass
            else:
                driver.find_element_by_xpath(comment).click()
        except:
            pass


    hotel_good_review_all= hotel_good_review_all.reset_index(drop=True) 
    hotel_bad_review_all= hotel_bad_review_all.reset_index(drop=True)
    hotel_good_bad_comment = pd.concat([hotel_good_review_all,hotel_bad_review_all], axis=1)
    hotel_good_bad_comment['hotel_title'] =hotel_title

    hotel_all_comment = hotel_all_comment.append(hotel_good_bad_comment)
    hotel_all_comment = hotel_all_comment.drop_duplicates()
    driver.close()

In [ ]:
##飯店評論資料輸出
hotel_all_comment.to_csv("hotel_all_comment.txt", sep='\t', index=False)

# Hotel Score

In [ ]:
hotel_all_score =pd.DataFrame()
#len(hotel_info["hotel_ref"])
for i in range(3):
    try:
        url = str(hotel_info["hotel_ref"].iloc[i])
        hotel_title =str(hotel_info["hotel_title"].iloc[i])
        print(i)
        driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
        driver.maximize_window()   
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'lxml')
        hotel_score =pd.DataFrame()
        
        '''
        hotel review 
        '''
        hotel_review = pd.DataFrame(columns = ['hotel_review'])
        review = driver.find_element_by_xpath('//*[@id="show_reviews_tab"]').text
        hotel_review= hotel_review.append({'hotel_review':review}, ignore_index=True)
        hotel_review["hotel_review"] =hotel_review["hotel_review"].str.replace("住客評語", "")
        print(hotel_review)



        driver.find_element_by_xpath('//*[@id="show_reviews_tab"]').click()

        '''
        hotel review score
        '''
        hotel_total_review = pd.DataFrame(columns = ['hotel_total_review'])
        total_review =driver.find_element_by_xpath('//*[@id="review_list_score"]/div[1]/span/span[1]').text
        hotel_total_review = hotel_total_review.append({'hotel_total_review':total_review}, ignore_index=True)
        hotel_total_review["hotel_total_review"] =hotel_total_review["hotel_total_review"].str.replace("\n", "")
        print(total_review)


        
        '''
        hotel fac score
        '''
        hotel_fac_review_score = pd.DataFrame(columns = ['hotel_fac_review_score'])
        for element in soup.findAll("span", {"class": "c-score-bar__score"}):
            fac_review_score = element.text
            hotel_fac_review_score= hotel_fac_review_score.append({'hotel_fac_review_score':fac_review_score}, ignore_index=True)
            print("fac score:"+fac_review_score)
        #hotel_fac_review_score=hotel_fac_review_score.T
        
        '''
        hotel fac title
        '''
        
        hotel_fac_review_title = pd.DataFrame(columns = ['hotel_fac_review_title'])
        for element in soup.findAll("span", {"class": "c-score-bar__title"}):
            fac_review_title = element.text
            hotel_fac_review_title= hotel_fac_review_title.append({'hotel_fac_review_title':fac_review_title}, ignore_index=True)
            hotel_fac_review_title["hotel_fac_review_title"] =hotel_fac_review_title["hotel_fac_review_title"].str.replace("\xa0", "")
            print("fac title:"+fac_review_title)
        #hotel_fac_review_title=hotel_fac_review_title.T
           
    except:
        pass
    
    
    
    hotel_fac_all_score = pd.DataFrame()
    hotel_review=hotel_review.reset_index(drop=True)
    hotel_total_review=hotel_total_review.reset_index(drop=True)
    hotel_fac_all_score = pd.concat([hotel_fac_review_title.reset_index(drop=True), hotel_fac_review_score], axis=1)
    
    hotel_fac_all_score = hotel_fac_all_score.T
    hotel_fac_all_score.columns =hotel_fac_all_score.iloc[0]
    hotel_fac_all_score=hotel_fac_all_score.drop(hotel_fac_all_score.index[[0]])
    print(hotel_fac_all_score)
    
    
    hotel_score = pd.concat([hotel_total_review,hotel_review,hotel_fac_all_score.reset_index(drop=True)], axis=1)
    hotel_score['hotel_title'] =hotel_title 
    
    hotel_score= hotel_score.reset_index(drop=True)
    hotel_all_score =hotel_all_score.append(hotel_score, ignore_index=True)
    
    print(hotel_all_score)
    hotel_all_score = hotel_all_score.drop_duplicates()
    
    driver.close()
    

# Try Code

每一層程式所嘗試的程式碼

In [ ]:
#第一層
soup = BeautifulSoup(driver.page_source,'lxml')
'''
title
'''
hotel_title_crawler = soup.select('.sr-hotel__title')
hotel_title= pd.DataFrame(columns = ['hotel_title'])

for element in  hotel_title_crawler:
    title =element.text
    hotel_title = hotel_title.append({'hotel_title': title}, ignore_index=True)
    hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("\n", "")
    hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("在新視窗開啟", "")
    
'''
location
'''
location_crawler = soup.select('.sr_card_address_line a',class_ = "bui-link")
hotel_location = pd.DataFrame(columns = ['hotel_location'])
for element in  location_crawler:
    location =element.text
    hotel_location = hotel_location.append({'hotel_location': location}, ignore_index=True)
    hotel_location["hotel_location"] =hotel_location["hotel_location"].str.replace("\n", "")
    hotel_location["hotel_location"] =hotel_location["hotel_location"].str.replace("顯示在地圖上", "")
    

'''
#price
'''
hotel_price_crawler = soup.select('.prco-ltr-right-align-helper span',class_='bui-u-sr-only')
hotel_price = pd.DataFrame(columns = ['hotel_price'])
for element in  hotel_price_crawler:
    price =element.text
    hotel_price= hotel_price.append({'hotel_price':price}, ignore_index=True)
    hotel_price = hotel_price.astype(str)
    hotel_price = hotel_price[~hotel_price['hotel_price'].isin(['NT$'])]
    hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("\n", "")
    hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("TWD", "")
    hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace("房價", "")
    hotel_price["hotel_price"] =hotel_price["hotel_price"].str.replace(",", "")
    #hotel_price = hotel_price.astype(int)


'''
#Review
'''
hotel_review_crawler = soup.select('.bui-review-score__text',class_= 'bui-review-score__text')
hotel_review = pd.DataFrame(columns = ['hotel_review'])
for element in  hotel_review_crawler:
    review =element.text
    hotel_review = hotel_review.append({'hotel_review': review}, ignore_index=True)
    hotel_review["hotel_review"] =hotel_review["hotel_review"].str.replace("則評語", "")
    hotel_review["hotel_review"] =hotel_review["hotel_review"].str.replace(",", "")


'''
#Total Review
'''
hotel_total_review_crawler = soup.select('.bui-review-score__badge')
hotel_total_review = pd.DataFrame(columns = ['hotel_total_review'])
for element in  hotel_total_review_crawler:
    total_review =element.text
    hotel_total_review = hotel_total_review.append({'hotel_total_review': total_review}, ignore_index=True)

    
    
'''
#score Review
'''
hotel_score_review_crawler = soup.select('.bui-review-score__title')
hotel_score_review = pd.DataFrame(columns = ['hotel_score_review'])
for element in  hotel_score_review_crawler:
    score_review =element.text
    hotel_score_review = hotel_score_review.append({'hotel_score_review': score_review}, ignore_index=True)

    
    
'''
#free cancel
'''
hotel_cancel = pd.DataFrame(columns = ['hotel_cancel'])
for element in soup.findAll("sup", {"class": "sr_room_reward sr_room_reward--bold e2e-no-prepayment"}):
    cancel =element.text
    print(cancel)
    hotel_cancel = hotel_cancel.append({'hotel_cancel': cancel}, ignore_index=True) 
    
hotel_cancel

'''
#ROOM
'''
hotel_room_crawler = soup.select('.roomNameInner sup',class_="c-beds-configuration")
hotel_room = pd.DataFrame(columns = ['hotel_room'])
for element in  hotel_room_crawler:
    room =element.text
    hotel_room = hotel_room.append({'hotel_room': room}, ignore_index=True) 
    
'''
#Ref
'''
elems = driver.find_elements_by_xpath('//a[@href][@class="hotel_name_link url"]')
hotel_ref= pd.DataFrame(columns = ['hotel_ref'])
for elem in elems:
    ref = elem.get_attribute("href")
    print(ref)
    #hotel_ref = hotel_ref.append({ref}, ignore_index=True)

                                   
                                   
                                   
'''
#hotel_room_type
'''
    
hotel_room_type= pd.DataFrame(columns = ['hotel_room_type'])
for element in driver.find_elements_by_class_name('c-beds-configuration'):
    room_type =element.text
    print(room_type)
    hotel_room_type = hotel_room_type.append({'hotel_room_type': room_type}, ignore_index=True)  
    

In [ ]:
#第二層

driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
driver.maximize_window()   
driver.get('https://www.booking.com/hotel/tw/cao-wu-dao-wen-chuang-lu-guan.zh-tw.html?aid=304142&label=gen173nr-1FCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AEB6AEB-AECiAIBqAIDuALU9Nr1BcACAQ&sid=0a05b3ee790bf2ece567cc57304a50ec&all_sr_blocks=199921311_237798361_4_1_0&checkin=2020-06-01&checkout=2020-06-03&dest_id=-2637824&dest_type=city&group_adults=4&group_children=0&hapos=55&highlighted_blocks=199921311_237798361_4_1_0&hpos=5&no_rooms=1&req_adults=4&req_children=0&room1=A%2CA%2CA%2CA&sr_order=popularity&sr_pri_blocks=199921311_237798361_4_1_0__554600&srepoch=1589122114&srpvid=c31a6821993a003d&ucfs=1&from=searchresults;highlight_room=#hotelTmpl')
soup = BeautifulSoup(driver.page_source,'lxml')    


hotel_cancel = pd.DataFrame(columns = ['hotel_cancel'])
for element in soup.findAll("div", {"class": "bui-spacer--small"}):
    cancel =element.text
    print(cancel)
    hotel_cancel = hotel_cancel.append({'hotel_cancel': cancel}, ignore_index=True) 
    hotel_cancel["hotel_cancel"] =hotel_cancel["hotel_cancel"].str.replace("\n", ",")
    hotel_cancel["hotel_cancel"] =hotel_cancel["hotel_cancel"].iloc[0]
    
driver.close() 
hotel_cancel


driver = webdriver.Chrome(r"D:\下載\chromedriver.exe")
driver.maximize_window()   
driver.get('https://www.booking.com/hotel/tw/cao-wu-dao-wen-chuang-lu-guan.zh-tw.html?aid=304142&label=gen173nr-1FCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AEB6AEB-AECiAIBqAIDuALU9Nr1BcACAQ&sid=0a05b3ee790bf2ece567cc57304a50ec&all_sr_blocks=199921311_237798361_4_1_0&checkin=2020-06-01&checkout=2020-06-03&dest_id=-2637824&dest_type=city&group_adults=4&group_children=0&hapos=55&highlighted_blocks=199921311_237798361_4_1_0&hpos=5&no_rooms=1&req_adults=4&req_children=0&room1=A%2CA%2CA%2CA&sr_order=popularity&sr_pri_blocks=199921311_237798361_4_1_0__554600&srepoch=1589122114&srpvid=c31a6821993a003d&ucfs=1&from=searchresults;highlight_room=#hotelTmpl')
soup = BeautifulSoup(driver.page_source,'lxml')    



'''
title
'''
hotel_title= pd.DataFrame(columns = ['hotel_title'])

for element in  soup.findAll("a", {"id": "hp_hotel_name_reviews"}):
    title =element.text
    hotel_title = hotel_title.append({'hotel_title': title}, ignore_index=True)
    hotel_title["hotel_title"] =hotel_title["hotel_title"].str.replace("\n", "")
hotel_title =  hotel_title.reset_index(drop=True)

'''
#hotel_room_type
'''
  
hotel_room_type= pd.DataFrame(columns = ['hotel_room_type'])
for element in soup.findAll("span", {"class": "m-rs-bed-display__bed-type-name"}):
    room_type =element.text
    hotel_room_type = hotel_room_type.append({'hotel_room_type': room_type}, ignore_index=True)
    hotel_room_type["hotel_room_type"] =hotel_room_type["hotel_room_type"].str.replace("\n", "")
    hotel_room_type =hotel_room_type.drop_duplicates()
hotel_room_type = hotel_room_type.reset_index(drop=True)      

    

    
    
'''
#Hotel Main facilities Type
''' 
hotel_all_fac= pd.DataFrame()
hotel_main_fac = pd.DataFrame(columns = ['hotel_main_fac'])
hotel_main_fac_temp = pd.DataFrame(columns = ['hotel_main_fac'])
for element in soup.findAll("div", {"class": "bui-spacer--small"}):
    main_fac =element.text
    #print(cancel)
    hotel_main_fac_temp = hotel_main_fac_temp.append({'hotel_main_fac': main_fac}, ignore_index=True) 
    hotel_main_fac_temp["hotel_main_fac"] =hotel_main_fac_temp["hotel_main_fac"].str.replace("\n", " ") 
    
hotel_main_fac[['hotel_main_fac']]=hotel_main_fac['hotel_main_fac'].append(pd.DataFrame(hotel_main_fac_temp.iloc[0,:]))
hotel_main_fac = hotel_main_fac.reset_index(drop=True)
 
    
     
    
'''
#Hotel facilities Type
'''
hotel_facilities_crawler = soup.select('.hprt-facilities-others span',class_='hprt-facilities-facility')
#hotel_room_crawler = soup.select('.hprt-facilities-others span')
hotel_facilities= pd.DataFrame(columns = ['hotel_facilities'])
hotel_fac= pd.DataFrame(columns = ['hotel_facilities'])
for element in  hotel_facilities_crawler:
    facilities =element.text
    hotel_facilities = hotel_facilities.append({'hotel_facilities': facilities}, ignore_index=True)
    hotel_facilities["hotel_facilities"] =hotel_facilities["hotel_facilities"].str.replace("\n", "")
    hotel_facilities = hotel_facilities.drop_duplicates()
hotel_fac =hotel_fac.append({'hotel_facilities':hotel_facilities['hotel_facilities'].str.cat(sep=' ')}, ignore_index=True)
#hotel_facilities =hotel_facilities.sort_values(by="hotel_facilities").T
hotel_fac= hotel_fac.reset_index(drop=True) 


'''
Address
'''
hotel_address= pd.DataFrame(columns = ['hotel_address'])
for element in soup.findAll("span", {"class":"js-hp_address_subtitle"}):
    address =element.text
    hotel_address = hotel_address.append({'hotel_address': address}, ignore_index=True)
    hotel_address["hotel_address"] =hotel_address["hotel_address"].str.replace("\n", "")
hotel_address =  hotel_address.reset_index(drop=True)

hotel_all_fac= pd.concat([hotel_title,hotel_room_type,hotel_fac,hotel_main_fac], axis=1)    
#hotel_all_fac = hotel_all_fac.append()

driver.close()




'''
username = driver.find_elements_by_xpath('//*[@id="hprt-table"]/tbody/tr[1]/td[1]/div/div[4]/div/ul/li[1]')    
hotel_room= pd.DataFrame(columns = ['data-name-en'])
for elem in username:
    room = elem.get_attribute("data-name-en")
    hotel_room  = hotel_room.append({'data-name-en': room}, ignore_index=True)    
    
    
'''
    


In [ ]:
##hotel comment
driver.get('https://www.booking.com/hotel/tw/lailai.zh-tw.html?aid=304142&label=gen173nr-1FCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AEB6AEB-AECiAIBqAIDuALU9Nr1BcACAQ&sid=09dd0dda5ec0150c0e4eebfbc255a04d&all_sr_blocks=105492715_251639934_0_1_0&checkin=2020-06-01&checkout=2020-06-03&dest_id=-2637824&dest_type=city&group_adults=4&group_children=0&hapos=102&highlighted_blocks=105492715_251639934_0_1_0&hpos=2&no_rooms=1&req_adults=4&req_children=0&room1=A%2CA%2CA%2CA&sr_order=popularity&sr_pri_blocks=105492715_251639934_0_1_0__640000&srepoch=1589619991&srpvid=5243400beaeb0066&ucfs=1&from=searchresults;highlight_room=#hotelTmpl')

driver.find_element_by_xpath('//*[@id="show_reviews_tab"]').click()
soup = BeautifulSoup(driver.page_source,'lxml')
hotel_bad_review_all = pd.DataFrame(columns = ['hotel_bad_review'])
main_fac = []
for  j in range(1,3):
    
    print(j)
    try:
        
        soup = BeautifulSoup(driver.page_source,'lxml')
        comment = '//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[2]/div/div['+str(j+1)+']/a'
        
        
        
        hotel_bad_review = pd.DataFrame(columns = ['hotel_bad_review'])
        for element in soup.findAll("div", {"class": "c-review__row lalala"}):
            bad_review =element.text
            print("baddddddd"+bad_review)
            hotel_bad_review= hotel_bad_review.append({'hotel_bad_review':bad_review}, ignore_index=True)
            
            hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("\n", "")
            hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("不喜歡", "")
            hotel_bad_review["hotel_bad_review"] =hotel_bad_review["hotel_bad_review"].str.replace("·", "")
            
        hotel_bad_review= hotel_bad_review.reset_index(drop=True) 
        hotel_bad_review_all = hotel_bad_review_all.append(hotel_bad_review)
      
        
        
        if j == 5:
            pass
        else:
            driver.find_element_by_xpath(comment).click()
    except:
        pass